In [1]:
import numpy as np
import shutil
import os
from ase import io

In [2]:
def extract_last_stress_tensor(file_path):
    
    stress_tensors = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # 遍历文件中的每一行
        for i, line in enumerate(lines):
            if 'STRESS| Analytical stress tensor [GPa]' in line:
                # 找到 STRESS| Analytical stress tensor 的位置，开始解析应力矩阵
                stress_matrix = []
                for j in range(i + 2, i + 5):  # 取接下来的 3 行应力张量
                    row_data = lines[j].split()[2:]  # 取 x, y, z 列的值
                    stress_matrix.append([np.round(float(val), 11) for val in row_data])
                stress_tensors.append(np.array(stress_matrix))  # 将该矩阵加入到列表中
    if stress_tensors:
        return stress_tensors[-1]
    else:
        return None

file_path = 'C3S_CellOPT.out'  # 替换成你实际文件的路径
last_stress_tensor = extract_last_stress_tensor(file_path)
last_stress_tensor

FileNotFoundError: [Errno 2] No such file or directory: 'C3S_CellOPT.out'

In [19]:
def read_last_frame_cell_vectors(file_path):
    with open(file_path, 'r') as file:
        content = [line for line in file.readlines() if line.strip()]

    last_frame_line = content[-1]
    parts = last_frame_line.split()

    vector_a = [np.round(float(parts[2]), 10), np.round(float(parts[3]), 10), np.round(float(parts[4]), 10)]
    vector_b = [np.round(float(parts[5]), 10), np.round(float(parts[6]), 10), np.round(float(parts[7]), 10)]
    vector_c = [np.round(float(parts[8]), 10), np.round(float(parts[9]), 10), np.round(float(parts[10]), 10)]

    cell_vectors = np.array([vector_a, vector_b, vector_c])
    return cell_vectors
file_path = './C3S_CellOPT-1.cell'
last_frame_cell_vectors = read_last_frame_cell_vectors(file_path)
last_frame_cell_vectors

array([[ 1.22320878e+01,  0.00000000e+00,  0.00000000e+00],
       [ 1.00506000e-05,  1.41445700e+01,  0.00000000e+00],
       [-8.12799176e+00, -2.44980000e-06,  1.69863453e+01]])

In [30]:
def extract_last_frame(file_path):
    with open(file_path, 'r') as file:
        file_content = file.readlines()

    frame_start_index = None
    for i in range(len(file_content) - 1, -1, -1):
        if file_content[i].startswith(" i ="):
            frame_start_index = i
            break

    atoms_in_last_frame = file_content[frame_start_index + 1:]

    frame_end_index = None
    for i in range(len(atoms_in_last_frame)):
        if atoms_in_last_frame[i].startswith(" i ="):
            frame_end_index = i
            break
            
    if frame_end_index is not None:
        atoms_in_last_frame = atoms_in_last_frame[:frame_end_index]

    atom_data = []
    for line in atoms_in_last_frame:
        atom_info = line.split()
        atom_name = atom_info[0]
        coordinates = [float(coord) for coord in atom_info[1:]]
        atom_data.append((atom_name, coordinates))
    return atom_data


def save_xyz_file(atom_data, output_path):
    num_atoms = len(atom_data)
    with open(output_path, 'w') as file:
        file.write(f"{num_atoms}\n")
        file.write("Lattice\n")
        
        for atom in atom_data:
            atom_name = atom[0]
            coordinates = atom[1]
            formatted_coords = " ".join([f"{coord:.10f}" for coord in coordinates])
            file.write(f"{atom_name} {formatted_coords}\n")


file_path = './C3S_CellOPT-pos-1.xyz'
last_frame_atoms = extract_last_frame(file_path)

output_path = './coord.xyz'
save_xyz_file(last_frame_atoms, output_path)

In [35]:
import os
import shutil
import numpy as np

# 创建所需的文件夹结构
def create_folders(base_folders):
    for base_folder in base_folders:
        pos_folder = os.path.join(base_folder, "pos")
        neg_folder = os.path.join(base_folder, "neg")
        
        # 如果文件夹存在，清空其中内容
        if os.path.exists(base_folder):
            shutil.rmtree(base_folder)
        
        # 创建文件夹结构
        os.makedirs(pos_folder, exist_ok=True)
        os.makedirs(neg_folder, exist_ok=True)

# 读取并修改 CP2K 输入文件
def modify_cp2k_input(input_file_path, output_file_path, cell_vectors):
    with open(input_file_path, 'r') as file:
        input_lines = file.readlines()

    # 标志位，判断是否进入&CELL部分
    in_cell_section = False
    new_lines = []

    # 记录&CELL部分的缩进
    cell_indent = None

    for line in input_lines:
        # 检查是否进入&CELL部分
        if "&CELL" in line:
            in_cell_section = True
            cell_indent = len(line) - len(line.lstrip())  # 获取&CELL行的前导空格数
            new_lines.append(line)  # 保留原有的&CELL部分头
            continue

        # 如果进入了&CELL部分，并且当前行是A、B、C基础矢量行，进行修改
        if in_cell_section:
            if "&END CELL" in line:
                in_cell_section = False
                new_lines.append(line)
            else:
                # 忽略每行前的空格，检查是否是A、B、C行
                stripped_line = line.strip()
                if stripped_line.startswith("A"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}A  {cell_vectors[0,0]:.10f} {cell_vectors[0,1]:.10f} {cell_vectors[0,2]:.10f}\n")
                elif stripped_line.startswith("B"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}B  {cell_vectors[1,0]:.10f} {cell_vectors[1,1]:.10f} {cell_vectors[1,2]:.10f}\n")
                elif stripped_line.startswith("C"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}C  {cell_vectors[2,0]:.10f} {cell_vectors[2,1]:.10f} {cell_vectors[2,2]:.10f}\n")
                else:
                    new_lines.append(line)
        else:
            new_lines.append(line)

    # 将修改后的内容写入到新的输入文件
    with open(output_file_path, 'w') as output_file:
        output_file.writelines(new_lines)

# 示例输入：假设 last_frame_cell_vectors 是从最后一帧中提取的基础矢量
last_frame_cell_vectors = np.array([
    [10.0, 0.0, 0.0],
    [0.0, 10.0, 0.0],
    [0.0, 0.0, 10.0]
])

# 创建文件夹结构
base_folders = ["./epsilonxx", "./epsilonyy", "./epsilonzz", "./epsilonxy", "./epsilonyz", "./epsilonzx"]
create_folders(base_folders)

# 读取输入文件并生成修改后的文件
input_file_path = './C3S_CellOPT.inp'  # 上传的输入文件路径

# 遍历每个文件夹并输出修改后的输入文件
for folder in base_folders:
    # 输出文件路径
    pos_file_path = os.path.join(folder, "pos", "displace.inp")
    neg_file_path = os.path.join(folder, "neg", "displace.inp")
    
    # 修改输入文件并保存到pos文件夹
    modify_cp2k_input(input_file_path, pos_file_path, last_frame_cell_vectors)
    # 修改输入文件并保存到neg文件夹
    modify_cp2k_input(input_file_path, neg_file_path, last_frame_cell_vectors)

# 输出生成的文件路径列表
base_folders

['./epsilonxx',
 './epsilonyy',
 './epsilonzz',
 './epsilonxy',
 './epsilonyz',
 './epsilonzx']

In [37]:
# 复制指定的文件到12个文件夹中的pos和neg子文件夹
def copy_files_to_folders(files, base_folders):
    for folder in base_folders:
        for subfolder in ["pos", "neg"]:
            # 构造目标路径
            target_folder = os.path.join(folder, subfolder)
            
            # 确保目标文件夹存在
            if os.path.exists(target_folder):
                for file in files:
                    # 确保源文件存在
                    if os.path.exists(file):
                        # 复制文件到目标文件夹
                        shutil.copy(file, target_folder)
                    else:
                        print(f"Warning: {file} not found. Skipping file.")
            else:
                print(f"Warning: Target folder {target_folder} not found. Skipping.")

# 全局变量：文件路径
potential_file = 'POTENTIAL'  # 替换为实际文件名
basis_set_file = 'BASIS_MOLOPT'  # 替换为实际文件名
dftd3_dat = 'dftd3.dat'           # 替换为实际文件名

# 创建文件夹结构
base_folders = ["./epsilonxx", "./epsilonyy", "./epsilonzz", "./epsilonxy", "./epsilonyz", "./epsilonzx"]

# 文件列表
files_to_copy = [potential_file, basis_set_file, dftd3_dat]

# 调用函数进行文件复制
copy_files_to_folders(files_to_copy, base_folders)


In [3]:
import numpy as np
import shutil
from ase import io
import subprocess
import os
from pathlib import Path
import pandas as pd

num_cores = 52 
cp2k_exec_path = "/home/zkm/CP2K/cp2k-2024.3/exe/local/cp2k.psmp" 
cellopt_inp = "cellopt.inp"  
cellopt_out = "cellopt.out" 
cellopt_jobname = "cellopt"
geoopt_inp = "geoopt.inp"  
geoopt_out = "geoopt.out" 
geoopt_jobname = "geoopt"

potential_file = 'POTENTIAL'
basis_set_file = 'BASIS_MOLOPT'
dftd3_dat = 'dftd3.dat'

up = 0.01

base_folders = ["./epsilonxx", "./epsilonyy", "./epsilonzz", "./epsilonyz", "./epsilonxz", "./epsilonxy"]
base_path = Path.cwd()

def check_existing_results(cp2k_out, jobname):
    if os.path.exists(cp2k_out) and os.path.exists(f"{jobname}-1.cell") and os.path.exists(f"{jobname}-pos-1.xyz"):
        return True
    else:
        try:
            os.remove(cp2k_out)
            os.remove(f"{jobname}-1.cell")
            os.remove(f"{jobname}-pos-1.xyz")
        except FileNotFoundError:
            pass
        return False

def create_folders(base_folders):
    for base_folder in base_folders:
        pos_folder = os.path.join(base_folder, "pos")
        neg_folder = os.path.join(base_folder, "neg")
        
        if os.path.exists(base_folder):
            shutil.rmtree(base_folder)
        
        os.makedirs(pos_folder, exist_ok=True)
        os.makedirs(neg_folder, exist_ok=True)

def copy_files_to_folders(files, base_folders):
    for folder in base_folders:
        for subfolder in ["pos", "neg"]:
            target_folder = os.path.join(folder, subfolder)
            
            if os.path.exists(target_folder):
                for file in files:
                    if os.path.exists(file):
                        shutil.copy(file, target_folder)
                    else:
                        print(f"Warning: {file} not found. Skipping file.")
            else:
                print(f"Warning: Target folder {target_folder} not found. Skipping.")

def run_cp2k_optimization(cp2k_inp, cp2k_out, jobname):
    if check_existing_results(cp2k_out, jobname):
        print("This folder already contains the cell optimization results.")
        return
        
    command = [
        "mpirun", 
        "-n", str(num_cores), 
        cp2k_exec_path, 
        "-o", cp2k_out, 
        cp2k_inp
    ]
    
    try:
        print(f"Running CP2K with {num_cores} cores...")
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        print(f"CP2K run completed successfully. Output is saved to {cp2k_out}.")
        print("Standard Output:", result.stdout.decode())
        print("Standard Error:", result.stderr.decode())
    
    except subprocess.CalledProcessError as e:
        print(f"Error occurred during CP2K execution: {e}")
        print("Standard Output:", e.stdout.decode())
        print("Standard Error:", e.stderr.decode())

def extract_last_frame(file_path):
    with open(file_path, 'r') as file:
        file_content = file.readlines()

    frame_start_index = None
    for i in range(len(file_content) - 1, -1, -1):
        if file_content[i].startswith(" i ="):
            frame_start_index = i
            break

    atoms_in_last_frame = file_content[frame_start_index + 1:]

    frame_end_index = None
    for i in range(len(atoms_in_last_frame)):
        if atoms_in_last_frame[i].startswith(" i ="):
            frame_end_index = i
            break
            
    if frame_end_index is not None:
        atoms_in_last_frame = atoms_in_last_frame[:frame_end_index]

    atom_data = []
    for line in atoms_in_last_frame:
        atom_info = line.split()
        atom_name = atom_info[0]
        coordinates = [float(coord) for coord in atom_info[1:]]
        atom_data.append((atom_name, coordinates))
    return atom_data

def read_last_frame_cell_vectors(file_path):
    with open(file_path, 'r') as file:
        content = [line for line in file.readlines() if line.strip()]

    last_frame_line = content[-1]
    parts = last_frame_line.split()

    vector_a = [np.round(float(parts[2]), 10), np.round(float(parts[3]), 10), np.round(float(parts[4]), 10)]
    vector_b = [np.round(float(parts[5]), 10), np.round(float(parts[6]), 10), np.round(float(parts[7]), 10)]
    vector_c = [np.round(float(parts[8]), 10), np.round(float(parts[9]), 10), np.round(float(parts[10]), 10)]

    cell_vectors = np.array([vector_a, vector_b, vector_c])
    return cell_vectors

def save_xyz_file(atom_data, cell_vectors, output_path):
    num_atoms = len(atom_data)
    with open(output_path, 'w') as file:
        file.write(f"{num_atoms}\n")
        file.write(f'Lattice="{" ".join(map(str, cell_vectors.flatten()))}" Properties=species:S:1:pos:R:3 pbc="T T T"\n')
        
        for atom in atom_data:
            atom_name = atom[0]
            coordinates = atom[1]
            formatted_coords = " ".join([f"{coord:.10f}" for coord in coordinates])
            file.write(f"{atom_name} {formatted_coords}\n")

def extract_last_stress_tensor(file_path):
    
    stress_tensors = []
    
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        # 遍历文件中的每一行
        for i, line in enumerate(lines):
            if 'STRESS| Analytical stress tensor [GPa]' in line:
                # 找到 STRESS| Analytical stress tensor 的位置，开始解析应力矩阵
                stress_matrix = []
                for j in range(i + 2, i + 5):  # 取接下来的 3 行应力张量
                    row_data = lines[j].split()[2:]  # 取 x, y, z 列的值
                    stress_matrix.append([np.round(float(val), 11) for val in row_data])
                stress_tensors.append(np.array(stress_matrix))  # 将该矩阵加入到列表中
    if stress_tensors:
        return stress_tensors[-1]
    else:
        return None

def modify_cp2k_input(input_file_path, output_file_path, cell_vectors):
    with open(input_file_path, 'r') as file:
        input_lines = file.readlines()

    # 标志位，判断是否进入&CELL部分
    in_cell_section = False
    new_lines = []

    # 记录&CELL部分的缩进
    cell_indent = None

    for line in input_lines:
        # 检查是否进入&CELL部分
        if "&CELL" in line:
            in_cell_section = True
            cell_indent = len(line) - len(line.lstrip())  # 获取&CELL行的前导空格数
            new_lines.append(line)  # 保留原有的&CELL部分头
            continue

        # 如果进入了&CELL部分，并且当前行是A、B、C基础矢量行，进行修改
        if in_cell_section:
            if "&END CELL" in line:
                in_cell_section = False
                new_lines.append(line)
            else:
                # 忽略每行前的空格，检查是否是A、B、C行
                stripped_line = line.strip()
                if stripped_line.startswith("A"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}A  {cell_vectors[0,0]:.10f} {cell_vectors[0,1]:.10f} {cell_vectors[0,2]:.10f}\n")
                elif stripped_line.startswith("B"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}B  {cell_vectors[1,0]:.10f} {cell_vectors[1,1]:.10f} {cell_vectors[1,2]:.10f}\n")
                elif stripped_line.startswith("C"):
                    new_lines.append(f"{' ' * (cell_indent + 2)}C  {cell_vectors[2,0]:.10f} {cell_vectors[2,1]:.10f} {cell_vectors[2,2]:.10f}\n")
                else:
                    new_lines.append(line)
        else:
            new_lines.append(line)

    # 将修改后的内容写入到新的输入文件
    with open(output_file_path, 'w') as output_file:
        output_file.writelines(new_lines)

epsilon_values = {
    "./epsilonxx": np.array([[up, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]),
    "./epsilonyy": np.array([[0.0, 0.0, 0.0], [0.0, up, 0.0], [0.0, 0.0, 0.0]]),
    "./epsilonzz": np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, up]]),
    "./epsilonyz": np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, up, 0.0]]),
    "./epsilonxz": np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [up, 0.0, 0.0]]),
    "./epsilonxy": np.array([[0.0, 0.0, 0.0], [up, 0.0, 0.0], [0.0, 0.0, 0.0]]),
}

def process_epsilon(folder, epsilon, cell_vectors_cellopt, input_file_path):
    cell_vectors_disp_pos = np.round(np.dot(cell_vectors_cellopt, (np.eye(3) + epsilon)), 10)
    cell_vectors_disp_neg = np.round(np.dot(cell_vectors_cellopt, (np.eye(3) - epsilon)), 10)
    
    pos_file_path = os.path.join(folder, "pos", geoopt_inp)
    neg_file_path = os.path.join(folder, "neg", geoopt_inp)
    
    modify_cp2k_input(input_file_path, pos_file_path, cell_vectors_disp_pos)
    modify_cp2k_input(input_file_path, neg_file_path, cell_vectors_disp_neg)

    output_xyz_pos = os.path.join(folder, "pos", "coord.xyz")
    output_xyz_neg = os.path.join(folder, "neg", "coord.xyz")
    remap_xyz(cell_vectors_disp_pos, "./coord_cellopt.xyz", output_xyz_pos)
    remap_xyz(cell_vectors_disp_neg, "./coord_cellopt.xyz", output_xyz_neg)

def remap_xyz(cell_vectors, coord_file, output_file):
    atoms = io.read(coord_file)
    atoms.set_cell(cell_vectors, scale_atoms=False)
    atoms.wrap()
    atoms.positions = np.round(atoms.positions, 10)
    atoms.write(output_file)

def symmetrize_and_save(array, filename="Cij.dat"):
    if array.ndim != 2 or array.shape[0] != array.shape[1]:
        raise ValueError("Cij have to be a (2D square array)!")
    symm_array = (array + array.T) / 2
    np.fill_diagonal(symm_array, np.diag(array))
    np.savetxt(filename, symm_array, fmt='%.3f')
    return symm_array


In [5]:
Cij = np.zeros((6, 6))
cellopt_inp = "cellopt.inp"  
cellopt_out = "cellopt.out" 
cellopt_jobname = "cellopt"
geoopt_inp = "geoopt.inp"  
geoopt_out = "geoopt.out" 
geoopt_jobname = "geoopt"
up = 0.01

for index, folder in enumerate(base_folders):
    os.chdir(base_path)
    stress_tensor_cellopt = extract_last_stress_tensor(cellopt_out)
    
    os.chdir(os.path.join(base_path, folder, "pos"))
    stress_tensor = extract_last_stress_tensor(geoopt_out)
    print(stress_tensor)
    Cn1_pos = - (stress_tensor[0,0] - stress_tensor_cellopt[0,0]) / (up)
    Cn2_pos = - (stress_tensor[1,1] - stress_tensor_cellopt[1,1]) / (up)
    Cn3_pos = - (stress_tensor[2,2] - stress_tensor_cellopt[2,2]) / (up)
    Cn4_pos = - (stress_tensor[1,2] - stress_tensor_cellopt[1,2]) / (up) #yz
    Cn5_pos = - (stress_tensor[0,2] - stress_tensor_cellopt[0,2]) / (up) #xz
    Cn6_pos = - (stress_tensor[0,1] - stress_tensor_cellopt[0,1]) / (up) #xy
    print(Cn1_pos)
    
    os.chdir(os.path.join(base_path, folder, "neg"))
    stress_tensor = extract_last_stress_tensor(geoopt_out)
    print(stress_tensor)
    Cn1_neg = (stress_tensor[0,0] - stress_tensor_cellopt[0,0]) / (up)
    Cn2_neg = (stress_tensor[1,1] - stress_tensor_cellopt[1,1]) / (up)
    Cn3_neg = (stress_tensor[2,2] - stress_tensor_cellopt[2,2]) / (up)
    Cn4_neg = (stress_tensor[1,2] - stress_tensor_cellopt[1,2]) / (up) #yz
    Cn5_neg = (stress_tensor[0,2] - stress_tensor_cellopt[0,2]) / (up) #xz
    Cn6_neg = (stress_tensor[0,1] - stress_tensor_cellopt[0,1]) / (up) #xy
    print(Cn1_neg)

    Cn1 =  np.round((Cn1_pos + Cn1_neg) / 2, 3)
    Cn2 =  np.round((Cn2_pos + Cn2_neg) / 2, 3)
    Cn3 =  np.round((Cn3_pos + Cn3_neg) / 2, 3)
    Cn4 =  np.round((Cn4_pos + Cn4_neg) / 2, 3)
    Cn5 =  np.round((Cn5_pos + Cn5_neg) / 2, 3)
    Cn6 =  np.round((Cn6_pos + Cn6_neg) / 2, 3)
    Cij[:, index] = [Cn1, Cn2, Cn3, Cn4, Cn5, Cn6]
    print(Cij)

[[-1.22716146 -0.02974587 -0.02476658]
 [-0.02974587 -0.62443084 -0.00548213]
 [-0.02476658 -0.00548213 -0.47545665]]
123.012959552
[[ 1.05177182  0.09406071 -0.05785465]
 [ 0.09406071  0.21166508  0.03805441]
 [-0.05785465  0.03805441  0.34118594]]
104.88036837099999
[[113.947   0.      0.      0.      0.      0.   ]
 [ 41.805   0.      0.      0.      0.      0.   ]
 [ 40.832   0.      0.      0.      0.      0.   ]
 [  2.177   0.      0.      0.      0.      0.   ]
 [ -1.654   0.      0.      0.      0.      0.   ]
 [  6.19    0.      0.      0.      0.      0.   ]]
[[-0.47699409  0.13927254  0.02830674]
 [ 0.13927254 -1.43247024  0.00441173]
 [ 0.02830674  0.00441173 -0.50778097]]
47.996222406
[[ 0.35119438 -0.0278565  -0.11891954]
 [-0.0278565   1.06410149  0.04611529]
 [-0.11891954  0.04611529  0.36687405]]
34.822623966
[[113.947  41.409   0.      0.      0.      0.   ]
 [ 41.805 124.829   0.      0.      0.      0.   ]
 [ 40.832  43.733   0.      0.      0.      0.   ]
 [  2.177

In [11]:
import numpy as np
Cij = np.zeros((6, 6))
new_values = [1, 2, 3, 4, 5, 6]

# 向第 2 列（索引为 1）添加值
Cij[:, 0] = new_values
Cij
base_folders = ["./epsilonxx", "./epsilonyy", "./epsilonzz", "./epsilonyz", "./epsilonxz", "./epsilonxy"]
for index, folder in enumerate(base_folders):
    print(index, folders)

0 ./epsilonxy
1 ./epsilonxy
2 ./epsilonxy
3 ./epsilonxy
4 ./epsilonxy
5 ./epsilonxy


In [50]:
import numpy as np

# 示例：创建一个非对称的 3x3 数组
array = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

# 提取上三角部分（不包括对角线）
upper_triangle = np.triu(array, k=1)

# 提取下三角部分（不包括对角线）
lower_triangle = np.tril(array, k=-1)

# 对称化：将上三角和下三角部分相加并取平均值
symmetric_array = (upper_triangle + lower_triangle.T) / 2

# 将对称化后的部分加回原始数组
symmetric_array += np.tril(array, k=0)  # 保持下三角和对角线不变


print("原始数组:")
print(array)
print("\n对称化后的数组:")
print(symmetric_array)

df = pd.DataFrame(array)
print(df.to_string(index=False, header=False))

原始数组:
[[1 2 3]
 [4 5 6]
 [7 8 9]]

对称化后的数组:
[[1. 3. 5.]
 [4. 5. 7.]
 [7. 8. 9.]]
1 2 3
4 5 6
7 8 9


In [42]:
import numpy as np

array = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
os.chdir(base_path)
def symmetrize_and_save(array, filename="Cij.dat"):
    if array.ndim != 2 or array.shape[0] != array.shape[1]:
        raise ValueError("Cij have to be a (2D square array)!")
    symm_array = (array + array.T) / 2
    np.fill_diagonal(symm_array, np.diag(array))
    np.savetxt(filename, symm_array, fmt='%.3f')
symmetrize_and_save(array)